# 듀얼 모멘텀 ( 절대 + 상대) 

In [1]:
def data_preprocessing(read_df, base_date):   
    read_df = read_df[read_df['Date'] >= base_date].copy()
    read_df.reset_index(inplace=True, drop = True)
    read_df['STD_YM'] = read_df['Date'].map(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m'))
    read_df['1M_RET'] = 0.0
    ym_keys = list(read_df['STD_YM'].unique())
    return read_df, ym_keys

def create_trade_book(sample, sample_codes):
    book = pd.DataFrame()
    book = sample[sample_codes].copy()
    book['STD_YM'] = book.index.map(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m'))
    for c in sample_codes:
        book['p '+c] = ''
        book['r '+c] = ''
    return book

# 상대모멘텀 Positioning
def positions(book, s_codes):
    std_ym = ''
    buy_phase = False
    for s in s_codes : 
        print(s)
        for i in book.index:
            if book.loc[i,'p '+s] == '' and book.shift(1).loc[i,'p '+s] == 'ready ' + s:
                std_ym = book.loc[i,'STD_YM']
                buy_phase = True
            
            if book.loc[i,'p '+s] == '' and book.loc[i,'STD_YM'] == std_ym and buy_phase == True : 
                book.loc[i,'p '+s] = 'buy ' + s
            
            if book.loc[i,'p '+ s] == '' :
                std_ym = None
                buy_phase = False
    return book

def multi_returns(book, s_codes):
    # 손익 계산
    rtn = 1.0
    buy_dict = {}
    num = len(s_codes)
    sell_dict = {}
    
    for i in book.index:
        for s in s_codes:
            if book.loc[i, 'p ' + s] == 'buy '+ s and \
            book.shift(1).loc[i, 'p '+s] == 'ready '+s and \
            book.shift(2).loc[i, 'p '+s] == '' :     # long 진입
                buy_dict[s] = book.loc[i, s]
            elif book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i, 'p '+s] == 'buy '+ s:     # long 청산
                sell_dict[s] = book.loc[i, s]
                # 손익 계산
                rtn = (sell_dict[s] / buy_dict[s]) -1
                book.loc[i, 'r '+s] = rtn
                print('개별 청산일 : ',i,' 종목코드 : ', s , 'long 진입가격 : ', buy_dict[s], ' |  long 청산가격 : ', sell_dict[s], ' | return:', round(rtn * 100, 2),'%') # 수익률 계산.

            if book.loc[i, 'p '+ s] == '':     # zero position || long 청산.
                buy_dict[s] = 0.0
                sell_dict[s] = 0.0


    acc_rtn = 1.0        
    for i in book.index:
        rtn  = 0.0
        count = 0
        for s in s_codes:
            if book.loc[i, 'p '+ s] == '' and book.shift(1).loc[i,'p '+ s] == 'buy '+ s:  # 청산. 이떄 수익률이 나오니깐.
                count += 1
                rtn += book.loc[i, 'r '+s]
        if (rtn != 0.0) & (count != 0) :
            # 내가 후보로 삼은 universe에서  1/n 이라고 가정.
            acc_rtn *= (rtn /count )  + 1
            print('누적 청산일 : ',i,'청산 종목수 : ',count, \
                  '청산 수익률 : ',round((rtn /count),4),'누적 수익률 : ' ,round(acc_rtn, 4)) # 수익률 계산.
        book.loc[i,'acc_rtn'] = acc_rtn
    
    print ('누적 수익률 :', round(acc_rtn, 4))

# 1단계 종목별 모멘텀 처리.

In [2]:
import pandas as pd
import numpy as np
import datetime

read_df = pd.read_csv('./new_us_etf_stock.csv')

In [3]:
read_df.head()

,Date,Code,Open,High,Low,Close,Adj Close,Volume
0,1980-12-12,AAPL,0.513393,0.515625,0.513393,0.513393,0.410525,117258400.0
1,1980-12-15,AAPL,0.488839,0.488839,0.486607,0.486607,0.389106,43971200.0
2,1980-12-16,AAPL,0.453125,0.453125,0.450893,0.450893,0.360548,26432000.0
3,1980-12-17,AAPL,0.462054,0.464286,0.462054,0.462054,0.369472,21610400.0
4,1980-12-18,AAPL,0.475446,0.477679,0.475446,0.475446,0.380182,18362400.0


In [4]:
month_last_df = pd.DataFrame(columns=['Date','Code','Adj Close'])
month_last_ret_df = pd.DataFrame(columns=['Date','Code','1M_RET'])
price_df, ym_keys = data_preprocessing(read_df, base_date='2010-01-02')

In [5]:
s_codes = list(price_df['Code'].unique())

for s in s_codes:
    print(s)
    for ym in ym_keys:
        try : 
            # 절대 모멘텀 사용 종가
            month_last_df= month_last_df.append(price_df.loc[price_df[(price_df['STD_YM'] == ym) \
                                                                      & (price_df['Code'] == s)].index[-1]\
                                                             ,['Date','Code','Adj Close']\
                                                            ]
                                               )
            
            # 상대 모멘텀 사용 수익률
            m_ret = price_df.loc[price_df[(price_df['STD_YM'] == ym) & (price_df['Code'] == s)].index[-1] , 'Adj Close'] \
            / price_df.loc[price_df[(price_df['STD_YM'] == ym) & (price_df['Code'] == s)].index[0] , 'Adj Close']
            price_df.loc[(price_df['STD_YM'] == ym) & (price_df['Code'] == s), ['1M_RET']] = m_ret
            
            
            month_last_ret_df= month_last_ret_df.append(price_df.loc[price_df[(price_df['STD_YM'] == ym) \
                                                                      & (price_df['Code'] == s)].index[-1]\
                                                             ,['Date','Code','1M_RET']\
                                                            ]
                                               )
        except IndexError:
            pass

AAPL
AMZN
BND
GLD
SLV
GDX
GM
MSFT
SPY
USM
USO
WMT


# 2단계. 절대*상대모멘텀 filtering 하기. Signal list

In [7]:
month_last_df

,Date,Code,Adj Close
18,2010-01-29,AAPL,24.035734
37,2010-02-26,AAPL,25.607582
60,2010-03-31,AAPL,29.409555
81,2010-04-30,AAPL,32.674633
101,2010-05-28,AAPL,32.147762
...,...,...,...
28305,2019-02-28,WMT,97.939934
28326,2019-03-29,WMT,97.014641
28347,2019-04-30,WMT,102.296577
28369,2019-05-31,WMT,101.440002


In [10]:
# 12개월  절대 모멘텀 
bf_month = 12
month_last_df.reset_index(inplace=True,drop=True)
month_last_tmp_df = month_last_df.pivot('Date','Code','Adj Close').copy()
abs_momentum_filter =  month_last_tmp_df / month_last_tmp_df.shift(bf_month) -1
abs_momentum_filter = abs_momentum_filter.where(abs_momentum_filter > 0.0 , np.nan)
abs_momentum_filter.fillna(0,inplace=True)
abs_momentum_filter[abs_momentum_filter != 0] = 1


# 유니버스 내 상위 20% 상대 모멘텀
ratio = 0.2   # 상위 20%에 드는 종목들만.
month_last_ret_df.reset_index(inplace=True,drop=True)
month_last_ret_df = month_last_ret_df.pivot('Date','Code','1M_RET').copy()
rel_momentum_filter = month_last_ret_df.rank(axis=1, ascending=False, method="max", pct=True) # 투자종목 선택할 rank
rel_momentum_filter = rel_momentum_filter.where( rel_momentum_filter < ratio , np.nan)
rel_momentum_filter.fillna(0,inplace=True)
rel_momentum_filter[rel_momentum_filter != 0] = 1

#듀얼 모멘텀.
dual_mementum_filter = abs_momentum_filter * rel_momentum_filter # 절대 모멘텀 * 상대 모멘텀 => 듀얼 모멘텀

# 3단계. signal positioning,( positioning / trading)

In [11]:
# 사실상 trading 부분.
sig_dict = dict()
for date in dual_mementum_filter.index:
    ticker_list = list(dual_mementum_filter.loc[date,dual_mementum_filter.loc[date,:] >= 1.0].index)
    sig_dict[date] = ticker_list
    
stock_c_matrix = price_df.pivot('Date','Code','Adj Close').copy()
book = create_trade_book(stock_c_matrix, s_codes)
book['STD_YM'] = book.index.map(lambda x : datetime.datetime.strptime(x,'%Y-%m-%d').strftime('%Y-%m'))

#ready 표시
for date,values in sig_dict.items():
    for stock in values:
        book.loc[date,'p '+ stock] = 'ready ' + stock
        

# # trading 부분을 만들려고 하는데
book = positions(book, s_codes)

AAPL
AMZN
BND
GLD
SLV
GDX
GM
MSFT
SPY
USM
USO
WMT


# 4단계 수익률 확인하기.

In [12]:
multi_returns(book, s_codes)

개별 청산일 :  2011-03-01  종목코드 :  AAPL long 진입가격 :  43.179466  |  long 청산가격 :  43.715107  | return: 1.24 %
개별 청산일 :  2011-03-01  종목코드 :  WMT long 진입가격 :  45.384689  |  long 청산가격 :  41.952442  | return: -7.56 %
개별 청산일 :  2011-04-01  종목코드 :  GDX long 진입가격 :  57.616322  |  long 청산가격 :  56.640263  | return: -1.69 %
개별 청산일 :  2011-05-02  종목코드 :  USM long 진입가격 :  44.592262  |  long 청산가격 :  41.984829  | return: -5.85 %
개별 청산일 :  2011-06-01  종목코드 :  GLD long 진입가격 :  150.41000400000001  |  long 청산가격 :  149.91000400000001  | return: -0.33 %
개별 청산일 :  2011-06-01  종목코드 :  SLV long 진입가격 :  33.869999  |  long 청산가격 :  35.75  | return: 5.55 %
개별 청산일 :  2011-07-01  종목코드 :  BND long 진입가격 :  66.480408  |  long 청산가격 :  65.94207800000001  | return: -0.81 %
개별 청산일 :  2011-07-01  종목코드 :  WMT long 진입가격 :  44.347271  |  long 청산가격 :  43.702071999999994  | return: -1.45 %
개별 청산일 :  2011-08-01  종목코드 :  AMZN long 진입가격 :  209.490005  |  long 청산가격 :  221.32000699999998  | return: 5.65 %
개별 청산일 :  2011-08-01  종목코드 :  MSF

In [18]:
book.loc['2011-01-30':'2011-02-01',:]  # 진입 포지션 확인

Code,AAPL,AMZN,BND,GLD,SLV,GDX,GM,MSFT,SPY,USM,...,r MSFT,p SPY,r SPY,p USM,r USM,p USO,r USO,p WMT,r WMT,acc_rtn
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-31,42.464890,169.639999,64.351532,129.869995,27.389999,51.096581,28.845346,22.431110,108.451126,41.804710,...,,,,,,,,ready WMT,,1.0
2011-02-01,43.179466,172.110001,64.227013,130.800003,27.870001,52.536987,28.813723,22.641428,110.187263,42.568073,...,,,,,,,,buy WMT,,1.0


In [19]:
book.loc['2011-02-26':'2011-03-01',:]  # 청산 포지션 확인

Code,AAPL,AMZN,BND,GLD,SLV,GDX,GM,MSFT,SPY,USM,...,r MSFT,p SPY,r SPY,p USM,r USM,p USO,r USO,p WMT,r WMT,acc_rtn
Date,,,,,,,,,,,,,,,,,,,,,
2011-02-28,44.203182,173.289993,64.540276,137.660004,33.110001,56.697117,26.505461,21.627945,112.218414,42.868267,...,,,,,,,,buy WMT,,1.00000
2011-03-01,43.715107,169.440002,64.509674,140.029999,33.869999,57.616322,26.046976,21.286194,110.347397,41.521667,...,,,,,,,,,-0.0756257,0.96839
